In [299]:
## IE 3311 Final Project
## Ishan Patel

In [301]:
# import necessary libraries
import json
import networkx as nx
import gurobipy as gp
from gurobipy import GRB

In [303]:
# Load KDP data from the location of the JSON file
with open('C:/Users/ishan/Desktop/OR Project/KDPlist.json', 'r') as file:
    data = json.load(file)

In [305]:
# Create a directed graph that represents donor-recipient relationships
G = nx.DiGraph()

In [307]:
# Add nodes with donor-recipient information
for idx, element in enumerate(data):
    G.add_node(idx, **element)

In [309]:
# creating a dictionary of compatibility rules based on blood types
DRC = {
    'O': ['O', 'A', 'B', 'AB'],
    'A': ['A', 'AB'],
    'B': ['B', 'AB'],
    'AB': ['AB']
}

In [311]:
# Add edges between nodes if both donor and recipient are compatible
for donor_node in G.nodes:
    donors = G.nodes[donor_node]['Donor']
    for donor in donors:
        for recipient_node in G.nodes:
            if donor_node == recipient_node:
                continue
            recipient = G.nodes[recipient_node]['Recipient']
            if recipient in DRC[donor]:
                G.add_edge(donor_node, recipient_node)

In [312]:
# Identify cycles of length 2
cycle_2 = []

for (i, j) in G.edges:# indicates that donor i is compatible with recipient j and could potentially donate a kidney.
    if (j, i) in G.edges:  # indicates that donor j is compatible with recipient i
        cycle_2.append((i, j))

In [313]:
# Identify cycles of length 3
potential_cycle_3 = []  # potential 3-way cycles
H = G.to_undirected()  

# identify potential 3-way cycles
for (u, v) in H.edges:
    for k in nx.common_neighbors(H, u, v): 
        potential_cycle_3.append((u, v, k)) 
# u is the first donor-recipient pair in the cycle
# v is the second donor-recipient pair in the cycle
# k is the third donor-recipient pair in the cycle

cycle_3 = []  # Final list of valid 3-way cycles
for (u, v, k) in potential_cycle_3:
    if (u, v) in G.edges and (v, k) in G.edges and (k, u) in G.edges:
        cycle_3.append((u, v, k)) 


In [314]:
# Optimization using Gurobi
model = gp.Model("KPD_Optimization")

In [356]:
# Define decision variables for cycles

# Binary variable: 1 if cycle is selected, 0 otherwise 
x_2 = {cycle: model.addVar(vtype=GRB.BINARY, name=f"x_2_{cycle}") for cycle in cycle_2} 

# Binary variable: 1 if cycle is selected, 0 otherwise  
x_3 = {cycle: model.addVar(vtype=GRB.BINARY, name=f"x_3_{cycle}") for cycle in cycle_3}

# Define compatibility scores
# Compatibility scores represent the likelihood of a successful transplant between donor `u` and recipient `v`
compatibility_scores = {(u, v): G[u][v].get('weight', 1) for u, v in G.edges}

# Objective function: Maximize compatibility scores
model.setObjective(
    gp.quicksum(x_2[cycle] for cycle in cycle_2) +
    gp.quicksum(x_3[cycle] for cycle in cycle_3),
    GRB.MAXIMIZE
)

In [358]:
# Constraint: Ensure no node is part of multiple cycles
# Each donor-recipient pair (node) can be in one cycle at most
# A node can be in one 2-way cycle or one 3-way cycle, but not both
for node in G.nodes:
    model.addConstr(
        gp.quicksum(x_2[cycle] for cycle in cycle_2 if node in cycle) + # Sum for 2-way cycles involving the node
       # Sum for 3-way cycles involving the node
        gp.quicksum(x_3[cycle] for cycle in cycle_3 if node in cycle) <= 1, # The sum is set to 1 so that way no node is selected in multiple cycles.
        name=f"Node_{node}_Participation"
    )

# After adding all constraints and variables
model.update()

In [323]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553316 - for non-commercial use only - registered to is___@ttu.edu
Optimize a model with 725 rows, 77700 columns and 155400 nonzeros
Model fingerprint: 0xdc5218a7
Variable types: 0 continuous, 77700 integer (77700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 185.0000000
Presolve removed 330 rows and 38850 columns
Presolve time: 0.16s
Presolved: 395 rows, 38850 columns, 77700 nonzeros
Found heuristic solution: objective 185.0000000
Variable types: 0 continuous, 38850 integer (38850 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    P

In [354]:
# Output solution
if model.status == GRB.OPTIMAL:
    print("You Did It! We Got A Pair!") # Print confirmation that a feasible solution was found
    
    # Print selected 2-way cycles
    print("\nSelected 2-Way Cycles:")
    for cycle in x_2:
        if x_2[cycle].x > 0.5:
            print(f"Cycle: {cycle[0]} & {cycle[1]} (Donor {cycle[0]} matches Recipient {cycle[1]})") 
    
    # Print selected 3-way cycles
    print("\nSelected 3-Way Cycles:")
    for cycle in x_3:
        if x_3[cycle].x > 0.5:
            print(f"Cycle: {cycle[0]} → {cycle[1]} → {cycle[2]} → {cycle[0]} "
                  f"(Donor {cycle[0]} matches Recipient {cycle[1]}, "
                  f"Donor {cycle[1]} matches Recipient {cycle[2]}, "
                  f"Donor {cycle[2]} matches Recipient {cycle[0]})")
else:
    print("No optimal solution found.")

You Did It! We Got A Pair!

Selected 2-Way Cycles:
Cycle: 0 & 9 (Donor 0 matches Recipient 9)
Cycle: 1 & 23 (Donor 1 matches Recipient 23)
Cycle: 4 & 5 (Donor 4 matches Recipient 5)
Cycle: 6 & 25 (Donor 6 matches Recipient 25)
Cycle: 12 & 13 (Donor 12 matches Recipient 13)
Cycle: 15 & 40 (Donor 15 matches Recipient 40)
Cycle: 16 & 47 (Donor 16 matches Recipient 47)
Cycle: 19 & 34 (Donor 19 matches Recipient 34)
Cycle: 24 & 53 (Donor 24 matches Recipient 53)
Cycle: 26 & 28 (Donor 26 matches Recipient 28)
Cycle: 27 & 41 (Donor 27 matches Recipient 41)
Cycle: 31 & 65 (Donor 31 matches Recipient 65)
Cycle: 33 & 44 (Donor 33 matches Recipient 44)
Cycle: 35 & 50 (Donor 35 matches Recipient 50)
Cycle: 36 & 60 (Donor 36 matches Recipient 60)
Cycle: 39 & 80 (Donor 39 matches Recipient 80)
Cycle: 48 & 96 (Donor 48 matches Recipient 96)
Cycle: 52 & 71 (Donor 52 matches Recipient 71)
Cycle: 54 & 64 (Donor 54 matches Recipient 64)
Cycle: 57 & 73 (Donor 57 matches Recipient 73)
Cycle: 61 & 75 (Donor